# Create some Fake Data

This can be used to run some validations about the logic before getting real customer data

In [14]:
import pandas as pd
import numpy as np
import json
from datetime import datetime, timedelta

from faker import Faker

#global variable to generate future data
today = pd.Timestamp('2030-12-31').date()

Create files in local file system in CSV format just for referece, as we will be writing those into Snowflake tables later


In [46]:
import random 

def generate_app_logs (n_users, n_days_ago, random_state=0, faker_state = 0):
    
    np.random.seed(random_state)
    fake = Faker()
    Faker.seed(faker_state)
    #set today to some date in the future so we will have plenty of new data going forward
    # today = pd.Timestamp('2030-12-31')
    # today = pd.Timestamp(datetime.now().date())
    
    employee_list = []
    app_list = []
    
    for user in range(1, n_users):
        user_email = fake.ascii_company_email()
        department = fake.company()
        division = fake.job()
        title = fake.job()
        
        has_leaved_company = np.random.uniform (0,100)
        if (has_leaved_company < 5): #Generate 5% people leaving the company 
            last_day_work_days = np.random.uniform (0, n_days_ago)
            field_last_work_day = today - timedelta (days = day)
        else:
            last_day_work_days = 0
            field_last_work_day = today + timedelta (days = 365) # set to the future for now
        
        n_logings = int(np.random.uniform (0, n_days_ago - last_day_work_days))  # Between never logging and every day
        
        for l in range (0, n_logings):
            day = np.random.uniform(0, n_days_ago) # get a ramdom day
            
            ## Let´s introduce some users who do not log in thet last 30 days
            not_loggin = np.random.uniform (0,100)
            if (not_loggin < 25):
                login_day = today - timedelta (days = day + last_day_work_days + 50)
            else:
                login_day = today - timedelta (days = day + last_day_work_days)
            
            app_id = random.choice([1,2])
            app_list.append([app_id, user_email, login_day])
            
        employee_list.append([app_id, user_email, department, division, title, field_last_work_day])
        
    df_employee = pd.DataFrame(employee_list, columns = ['app_id', 'session_user', 'department', 'division', 'title', 'last_day_of_work'])
    df_app = pd.DataFrame(app_list, columns = ['app_id', 'session_user', 'snapshot_datetime'])
    
    return df_app, df_employee
        
        

In [47]:

df_app_1, df_employee = generate_app_logs (n_users=5000, n_days_ago =3650, random_state= 6, faker_state= 4)

In [48]:
df_app_1.to_csv('../data/generated/sample_okta_logs.csv', index=False )

In [49]:
df_employee.to_csv('../data/generated/sample_employee_metadata.csv', index=False)

In [50]:
df_app_2, k = generate_app_logs (n_users=5000, n_days_ago =3650, random_state= 19, faker_state= 4)

In [51]:
df_app_2.to_csv('../data/generated/sample_app_logs.csv', index=False )

In [32]:
def generate_working_days (n_days_ago = 365):
    
    # today = pd.Timestamp(datetime.now().date())
    
    calendar_list = []
    
    for d in range(0, n_days_ago):    
        calendar_day = today - timedelta (days = d)
                
        num = np.random.uniform (0,100)
        if (num < 20): # 20% of holidays
            is_working_day = False
        else:
            is_working_day = True
        
        calendar_list.append([calendar_day, is_working_day])
        
    
    df_cal = pd.DataFrame(calendar_list, columns = ['snapshot_datetime', 'work_day'])
    
    return df_cal    

In [33]:
df_cal = generate_working_days(n_days_ago = 3650)


In [34]:
df_cal

,snapshot_datetime,work_day
0,2030-12-31,1
1,2030-12-30,0
2,2030-12-29,1
3,2030-12-28,0
4,2030-12-27,1
...,...,...
3645,2021-01-07,1
3646,2021-01-06,1
3647,2021-01-05,1
3648,2021-01-04,1


In [36]:
df_cal.to_csv('../data/generated/sample_work_days.csv', index=False)

Generate tables

In [37]:
from snowflake.snowpark.session import Session
import snowflake.snowpark.functions as F
import snowflake.snowpark.types as T

In [49]:
with open('../../config/creds_cc.json', 'r') as ff:
    conn_param=json.load(ff)

session = Session.builder.configs(conn_param).create() 

In [50]:
session.sql("create or replace database dev_snowpatrol").collect()
session.sql("create or replace schema main").collect()

session.use_database("dev_snowpatrol")
session.use_schema("dev_snowpatrol.main")

In [51]:
df = pd.read_csv('../../data/sample_okta_logs.csv')
df.columns = df.columns.str.upper()

table_name = 'SAMPLE_OKTA_LOGS'

session.write_pandas(df, table_name, auto_create_table=True)

In [52]:
df = pd.read_csv('../../data/sample_work_days.csv')
df.columns = df.columns.str.upper()

table_name = 'SAMPLE_WORK_DAYS'

session.write_pandas(df, table_name, auto_create_table=True)

In [53]:
df = pd.read_csv('../../data/sample_app_logs.csv')
df.columns = df.columns.str.upper()

table_name = 'SAMPLE_APP_LOGS'

session.write_pandas(df, table_name, auto_create_table=True)

In [54]:
df = pd.read_csv('../../data/sample_employee_metadata.csv')
df.columns = df.columns.str.upper()

table_name = 'SAMPLE_EMPLOYEE_METADATA'

session.write_pandas(df, table_name, auto_create_table=True)